# Objetivo

Avaliar se há clusters no portfólio 2. Em um segundo momento, a partir de uma clusterização global (na base toda), buscar novos clientes na base completa.

# Método

A partir do conjunto reduzido de features (47), aplicar um processo de clusterização (AgglomerativeClustering).

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import feather

## Leitura da lista de features a excluir

In [2]:
# obtida em EDA Pandas_profiling_sample_10k_market.ipynb
exclude_df = pd.read_csv('../exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)

143

## Carregamento Portfólio 1

In [3]:
p2 = pd.read_csv('../clientes3_merge.csv')
p2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(14), float64(144), int64(2), object(22)
memory usage: 351.5+ KB


## Redução no número de features

In [4]:
p2.drop(columns=exclude_variables, inplace= True)

In [5]:
p2.shape

(265, 39)

# Clusterização

## Dados do portfólio

Para usar nos dados do portfólio, temos que passar as strings para valor numérico.

In [6]:
# portfolio 2, reduzido (micro): p1micro
p2micro = p2.drop(columns=['Unnamed: 0', 'id'])

In [7]:
p2micro.dtypes == object

fl_matriz                                False
de_natureza_juridica                      True
sg_uf                                     True
natureza_juridica_macro                   True
de_ramo                                   True
setor                                     True
idade_empresa_anos                       False
idade_emp_cat                             True
fl_me                                    False
fl_sa                                    False
fl_mei                                   False
fl_ltda                                  False
dt_situacao                               True
fl_st_especial                           False
fl_email                                 False
fl_telefone                              False
fl_rm                                     True
nm_divisao                                True
nm_segmento                               True
fl_optante_simples                        True
fl_optante_simei                          True
sg_uf_matriz 

In [8]:
p2micro.head(3)

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
0,True,MUNICIPIO,MA,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,21.813699,> 20,False,False,...,OESTE MARANHENSE,PINDARE,True,NaN,NaN,NaN,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",141308380.0,144348380.0
1,True,SOCIEDADE SIMPLES LIMITADA,PI,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,16.389041,15 a 20,False,False,...,NORTE PIAUIENSE,BAIXO PARNAIBA PIAUIENSE,True,2.0,70.0,1.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",27818640.0,27818640.0
2,True,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,AM,ADMINISTRACAO PUBLICA,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,40.380822,> 20,False,False,...,CENTRO AMAZONENSE,MANAUS,True,NaN,NaN,NaN,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",92728800.0,92728800.0


In [9]:
# Vou começar com 4 features: de_natureza_juridica, sg_uf, de_ramo, de_nivel_atividade
categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo', 'de_nivel_atividade']
for feat in categorical_cols:
    print(p2micro[feat].value_counts())
    print('\n')

MUNICIPIO                                                                 108
SOCIEDADE EMPRESARIA LIMITADA                                              62
ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU DO DISTRITO FEDERAL           20
EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA     13
ORGAO PUBLICO DO PODER EXECUTIVO MUNICIPAL                                  9
ASSOCIACAO PRIVADA                                                          8
AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL                                   6
SERVICO SOCIAL AUTONOMO                                                     5
AUTARQUIA FEDERAL                                                           4
COOPERATIVA                                                                 4
FUNDACAO PUB DE DIREITO PUB EST OU DO DF                                    3
ORGAO PUBLICO DO PODER LEGISLATIVO ESTADUAL OU DO DISTRITO FEDERAL          3
EMPRESARIO INDIVIDUAL                                           

In [10]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [11]:
# apply le on categorical feature columns
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

#portfolio 1 com labelencoder: p1enc
p2enc=pd.DataFrame()
categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo']
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
# IMPORTANTE aqui, só três features, para poder visualizar em um gráfico 3D.
# Também porque de_nivel_atividade tem alguns NaN, que tem que ser resolvidos depois
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
p2enc[categorical_cols] = p2micro[categorical_cols].apply(lambda col: le.fit_transform(col))

p2enc[categorical_cols].head(5)

,de_natureza_juridica,sg_uf,de_ramo
0,14,2,0
1,24,3,7
2,15,1,0
3,14,3,0
4,14,4,0


In [12]:
p2enc.shape

(265, 3)

In [54]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, affinity='manhattan', linkage='average')
p2enc['cluster'] = cluster.fit_predict(p2enc)

In [55]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
p2enc['de_natureza_juridica']=p2enc['de_natureza_juridica'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['de_ramo']=p2enc['de_ramo'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['sg_uf']=p2enc['sg_uf'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc.head(5)

,de_natureza_juridica,sg_uf,de_ramo,cluster
0,13.589738,2.430442,0.670056,2
1,24.711912,2.366693,7.781228,1
2,14.866868,1.388102,-0.533243,2
3,14.225765,2.891853,-0.267718,2
4,13.122184,4.157033,-0.140448,2


In [56]:
import plotly.express as px
fig = px.scatter_3d(p2enc, x='de_natureza_juridica', y='sg_uf', z='de_ramo', color = 'cluster')
fig.show()

## Conclusão até o momento

Clusterização parece interessante. Observar que, das três categorias avaliadas, o `de_ramo` não teve muita influência, porque está bem polarizado em uma das classes. Próximos passos: incluir mais features (sabendo que vai impedir a visualização em mais de 3 dimensões) e aplicar redução de dimensionalidade (PCA? antes ou depois da clusterização?).